In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
from pyLDAvis import prepare
import string
import re
import nltk
import MeCab
nltk.download('wordnet')
import matplotlib.pyplot as plt  # max_iterの適切な値を見極めるために必要

In [ ]:
# ファイルからデータを読み込む
with open('text.txt', 'r', encoding='utf-8') as f:#テキストファイルの名前を入れる
    lines = f.readlines()

In [ ]:
# テキスト前処理を行う
exclude = set(string.punctuation)
m = MeCab.Tagger("-Owakati")  # MeCabを使って日本語テキストを分かち書き

In [ ]:
# 特定の文章を取り除く条件を設定
#keywords_to_remove = [" ", " "]

In [ ]:
# 特定の文章を取り除く
#lines = [line for line in lines if all(keyword not in line for keyword in keywords_to_remove)]

In [ ]:
def clean(doc):
    punc_free = ''.join(ch for ch in doc if ch not in exclude)
    no_digits = re.sub(r'\d+', '', punc_free)  # 数字を削除する
    no_alpha = re.sub(r'[a-zA-Z]+', '', no_digits)  # 英字を削除する
    normalized = m.parse(no_alpha).strip()
    return normalized
lines = [clean(line) for line in lines]

In [ ]:
# 文書-単語行列の作成
vect = CountVectorizer(max_df=1.0, stop_words=None)#1.0なら全て
data = vect.fit_transform(lines)

In [ ]:
# 最適なパラメータでLDAの適用と学習
lda = LatentDirichletAllocation(n_components=8,#トピックの数
                                max_iter=40,
                                learning_method='online',
                                learning_offset=10,#デフォルト
                                random_state=0)

lda.fit(data)

In [ ]:
# pyLDAvisを用いた可視化
pyLDAvis.enable_notebook()

In [ ]:
# 必要なデータを取得
topic_term_dists = lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis]  # 各トピックの単語分布
doc_topic_dists = lda.transform(data)  # 各文書のトピック分布
term_frequency = data.sum(axis=0).A1  # 各単語の出現回数
vocab = vect.get_feature_names_out()  # 単語リスト
doc_lengths = data.sum(axis=1).A1  # 各文書の単語数

In [ ]:
# pyLDAvis.prepare 関数でデータを可視化形式に変換
panel = pyLDAvis.prepare(
    topic_term_dists=topic_term_dists,
    doc_topic_dists=doc_topic_dists,
    doc_lengths=doc_lengths,
    vocab=vocab,
    term_frequency=term_frequency,
    mds='tsne'
)

In [ ]:
# 結果を表示
panel

In [ ]:
# ここからは任意

In [ ]:
#最適なlearning_offsetの値をグラフから探る
likelihoods = []
max_iters_range = range(1, 50)#適切な値を探る

for max_iter in max_iters_range:
    lda = LatentDirichletAllocation(n_components=8, max_iter=max_iter,
                                    learning_method='online',
                                    learning_offset=10,
                                    random_state=0)
    lda.fit(data)
    likelihood = lda.score(data)
    likelihoods.append(likelihood)

# 学習曲線をプロット
plt.plot(max_iters_range, likelihoods, marker='o')
plt.xlabel('max_iter')
plt.ylabel('Log Likelihood')
plt.title('LDA Learning Curve')
plt.show()